In [14]:
import json
import re
import requests
from subprocess import run, CalledProcessError
from collections import defaultdict
import runpy
import tempfile

In [4]:
data = json.loads(requests.get('https://storage.googleapis.com/flutter_infra_release/releases/releases_linux.json').text)
flutter_versions = data['releases']

In [ ]:
# replace these
flutter_repo_path = '../../co/sdk/flutter'
engine_repo_path = '../../co/sdk/engine'
dart_repo_path = '../../co/sdk/dart-sdk/full-clone'

def make_cache(fn):
    d = {}
    def caching_fn(x):
        if x not in d:
            d[x] = fn(x)
        return d[x]
    return caching_fn

def get_engine_version(flutter_commit):
    try:
        run(['git', 'show', flutter_commit + ':engine/README.md'], check=True, capture_output=True, cwd=flutter_repo_path)
        return True, flutter_commit
    except CalledProcessError:
        pass
    p = run(['git', 'show', flutter_commit + ':bin/internal/engine.version'], check=True, capture_output=True, cwd=flutter_repo_path)
    return False, p.stdout.decode().strip()

def get_dart_version(engine_commit):
    new, engine_commit = engine_commit
    contents = run(['git', 'show', engine_commit + ':DEPS'], check=True, capture_output=True, cwd=flutter_repo_path if new else engine_repo_path).stdout
    m = re.search(r"'dart_revision': '([a-fA-F0-9]+)'", contents.decode())
    return m.group(1)

sv_template_file = tempfile.NamedTemporaryFile()
sv_template_file.write('{{SNAPSHOT_HASH}}'.encode())
sv_template_file.flush()
sv_out_file = tempfile.NamedTemporaryFile()

def get_snapshot_hash(dart_commit):
    run(['git', 'checkout', dart_commit], check=True, cwd=dart_repo_path)
    sv_out_file.seek(0)
    run(['python3', 'tools/make_version.py', '--input=' + sv_template_file.name, '--output=' + sv_out_file.name], check=True, cwd=dart_repo_path)
    return sv_out_file.read().decode()

get_engine_version = make_cache(get_engine_version)
get_dart_version = make_cache(get_dart_version)
get_snapshot_hash = make_cache(get_snapshot_hash)

#get_engine_version('master')
#get_dart_version('11d756a62ed0ddf87a9ce20b219b55300ec6b67d')
#get_snapshot_version('06536d68ca0f27528b0bf729f4b8d673ed14beda')

In [11]:
for repo in [flutter_repo_path, engine_repo_path, dart_repo_path]:
    print('fetching', repo)
    run(['git', 'fetch'], check=True, cwd=repo)

fetching ../../co/sdk/flutter


From github.com:flutter/flutter
 * [new branch]              gh-readonly-queue/master/pr-181841-779bad15af4594346cf310d6958a8819d4c002ae -> origin/gh-readonly-queue/master/pr-181841-779bad15af4594346cf310d6958a8819d4c002ae


fetching ../../co/sdk/engine
fetching ../../co/sdk/dart-sdk/full-clone


From github.com:dart-lang/sdk
   e88f7377bab..56294a92d5c  dev              -> origin/dev
   e88f7377bab..56294a92d5c  lkgr-dev         -> origin/lkgr-dev
 * [new tag]                 3.12.0-105.0.dev -> 3.12.0-105.0.dev


In [ ]:
skipped_commits = { '8b9b90e75107181aadc303d8d7422205863a35dd', 'f732038a8cf4562ce38a1d7debb30209ae3da896' }

In [26]:
print('| Release date | Channel | Version | Commit | Engine commit | Dart SDK commit | Snapshot version |')
print('| ------------ | ------- | ------- | ------ | ------------- | --------------- | ---------------- |')
for v in sorted(flutter_versions, key=lambda x: x['release_date'], reverse=True):
    if v['channel'] not in { 'stable', 'beta' }: continue
    if v['hash'] in skipped_commits: continue
    engine_commit = get_engine_version(v['hash'])
    dart_commit = get_dart_version(engine_commit)
    snapshot_hash = get_snapshot_hash(dart_commit)
    engine_commit = '' if engine_commit[0] else engine_commit[1]
    print(f"| {v['release_date'][:10]} | {v['channel']} | {v['version']} | {v['hash']} | {engine_commit} | {dart_commit} | {snapshot_hash} |")

| Release date | Channel | Version | Commit | Engine commit | Dart SDK commit | Snapshot version |
| ------------ | ------- | ------- | ------ | ------------- | --------------- | ---------------- |
| 2026-01-29 | stable | 3.38.9 | 67323de285b00232883f53b84095eb72be97d35c |  | f934176b0195e655c73e92d431079457ceba488f | 1ce86630892e2dca9a8543fdb8ed8e22 |
| 2026-01-29 | beta | 3.41.0-0.2.pre | a7c4c6bf74749e4562e9f47c762a815201285132 |  | bc20d999d68a261ae12c56bab7ba304bd0de5346 | 78da37fed6bf1489361a312568249f3f |
| 2026-01-27 | stable | 3.38.8 | bd7a4a6b5576630823ca344e3e684c53aa1a0f46 |  | 2da4111d8d0cbf2a83c0662251508f017000da8a | 1ce86630892e2dca9a8543fdb8ed8e22 |
| 2026-01-26 | beta | 3.41.0-0.1.pre | be9275b8229b50e19bff4694643c39bb71401010 |  | bc20d999d68a261ae12c56bab7ba304bd0de5346 | 78da37fed6bf1489361a312568249f3f |
| 2026-01-14 | beta | 3.41.0-0.0.pre | ee37b97405045145783cd6df94d0dedf10438554 |  | fb9aa1719c55eb84dcd573c355e59611b56bc23d | 78da37fed6bf1489361a312568249f3f |